# Project ETL


In [1]:
import pymongo
import time
import requests
import json
import pandas as pd

from config import api_key, gkey

from yelpapi import YelpAPI
yelp_api = YelpAPI(api_key)

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

### Extracting restaurant data using Yelp API 

In [2]:
#Retrieve 50 for each term
term = 'restaurant'
search_limit = 50
latitude = 33.69173830884756
longitude = -118.01170349121094
radius = 10000
responses = yelp_api.search_query(term = term,
                                 limit = search_limit,
                                 latitude = latitude,
                                 longitude = longitude,
                                 radius = radius
                                )

term = 'fast food'
search_limit = 50
latitude = 33.69173830884756
longitude = -118.01170349121094
radius = 10000
responses2 = yelp_api.search_query(term = term,
                                 limit = search_limit,
                                 latitude = latitude,
                                 longitude = longitude,
                                 radius = radius
                                )


In [3]:
#Creates  Dataframe data_businesses
cols = list(responses['businesses'][0].keys())
data_businesses = pd.DataFrame(columns=cols)


In [4]:
#Insert first 50 restaurants into the data_businesses Dataframe
for biz in responses['businesses']:
    data_businesses = data_businesses.append(biz, ignore_index=True)
#Insert first 50 fast food restaurants into the data_businesses Dataframe
for biz in responses2['businesses']:
    data_businesses = data_businesses.append(biz, ignore_index=True)


In [5]:
data_businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Z0aogEeyM9iXI6lz-Wo_Fg,dukes-huntington-beach-huntington-beach-2,Duke's Huntington Beach,https://s3-media2.fl.yelpcdn.com/bphoto/_KInBr...,False,https://www.yelp.com/biz/dukes-huntington-beac...,1932,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 33.656629, 'longitude': -118.002219}","[pickup, delivery]",$$,"{'address1': '317 Pacific Coast Hwy', 'address...",+17143746446,(714) 374-6446,4004.463159
1,ElvZ-9U8y_LtJvVQ3oUibg,heirloom-a-modern-farmhouse-huntington-beach,Heirloom a Modern Farmhouse,https://s3-media3.fl.yelpcdn.com/bphoto/jam7ve...,False,https://www.yelp.com/biz/heirloom-a-modern-far...,243,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 33.69601, 'longitude': -117.98834}",[delivery],$$,"{'address1': '18344 Beach Blvd', 'address2': N...",+17143756543,(714) 375-6543,2231.854979
2,swJs0EEKdj3JF6XzccG0Nw,offshore-9-rooftop-lounge-huntington-beach,Offshore 9 Rooftop Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/gsaRYX...,False,https://www.yelp.com/biz/offshore-9-rooftop-lo...,109,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 33.6523, 'longitude': -117.99353}",[delivery],NaN,"{'address1': '21100 Pacific Coast Hwy', 'addre...",+17148458000,(714) 845-8000,4696.751440
3,TqhuzJxO3A6jgf7JHGbBng,mamas-on-39-huntington-beach,Mama's On 39,https://s3-media3.fl.yelpcdn.com/bphoto/4PK-bw...,False,https://www.yelp.com/biz/mamas-on-39-huntingto...,3461,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 33.65734, 'longitude': -117.98711}","[pickup, delivery]",$$,"{'address1': '21022 Beach Blvd', 'address2': '...",+17143741166,(714) 374-1166,4453.139520
4,HtS8m3k06JEXHbjdGLZAkQ,watertable-huntington-beach,Watertable,https://s3-media3.fl.yelpcdn.com/bphoto/jbtPm-...,False,https://www.yelp.com/biz/watertable-huntington...,610,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 33.651300643275626, 'longitude': ...",[delivery],$$$,"{'address1': '21500 Pacific Coast Hwy', 'addre...",+17148454776,(714) 845-4776,4897.321363


### Transforming the Yelp data

In [6]:
#drop duplicate data
data_businesses.drop_duplicates(subset ="name",keep = False, inplace = True)

In [7]:
#remove some columns
data_businesses = data_businesses.drop(['alias','image_url','url','coordinates','display_phone','distance'], axis=1)


In [8]:
#reset the index
data_businesses = data_businesses.reset_index(drop=True)


In [9]:
data_businesses.head()

,id,name,is_closed,review_count,categories,rating,transactions,price,location,phone
0,Z0aogEeyM9iXI6lz-Wo_Fg,Duke's Huntington Beach,False,1932,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"[pickup, delivery]",$$,"{'address1': '317 Pacific Coast Hwy', 'address...",+17143746446
1,ElvZ-9U8y_LtJvVQ3oUibg,Heirloom a Modern Farmhouse,False,243,"[{'alias': 'newamerican', 'title': 'American (...",4.5,[delivery],$$,"{'address1': '18344 Beach Blvd', 'address2': N...",+17143756543
2,swJs0EEKdj3JF6XzccG0Nw,Offshore 9 Rooftop Lounge,False,109,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,[delivery],NaN,"{'address1': '21100 Pacific Coast Hwy', 'addre...",+17148458000
3,TqhuzJxO3A6jgf7JHGbBng,Mama's On 39,False,3461,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"[pickup, delivery]",$$,"{'address1': '21022 Beach Blvd', 'address2': '...",+17143741166
4,HtS8m3k06JEXHbjdGLZAkQ,Watertable,False,610,"[{'alias': 'newamerican', 'title': 'American (...",4.5,[delivery],$$$,"{'address1': '21500 Pacific Coast Hwy', 'addre...",+17148454776


### Loading Yelp data into MongoDB

In [10]:
#Databse restaurants_db
db = client.restaurants_db
#Collection business
business = db.business

#Insert Dataframe data_businesses into MongoDB 
records = json.loads(data_businesses.T.to_json()).values()
db.business.insert_many(records)


### Yelp - another API call to extract the reviews
    

In [11]:
#Return 3 reviews for each restaurant
first_time = True

for index, row in data_businesses.iterrows():
    id_business =  row['id']
    responses_review = yelp_api.reviews_query(id = id_business)
    if first_time:
        cols = list(responses_review['reviews'][0].keys())
        cols.append('id_reviews')
        data_reviews = pd.DataFrame(columns=cols)
        first_time = False

    
    for resp in responses_review['reviews']:
        my_row = {"id":id_business, 
                "url":resp['url'],
                "text":resp['text'],
                "rating":resp['rating'],
                "time_created":resp['time_created'],
                "user":resp['user'],
                "id_reviews":resp['id']}                    
        data_reviews=data_reviews.append(my_row, ignore_index=True)
        my_row.clear()
 

In [12]:
data_reviews.head()

,id,url,text,rating,time_created,user,id_reviews
0,Z0aogEeyM9iXI6lz-Wo_Fg,https://www.yelp.com/biz/dukes-huntington-beac...,I have never had a better bar tender than Jame...,5,2021-04-26 16:34:28,"{'id': '9oWEPTd5dI36u1Zf6_ONlw', 'profile_url'...",EeQ1jhQtEV3dMIPGP4cU6w
1,Z0aogEeyM9iXI6lz-Wo_Fg,https://www.yelp.com/biz/dukes-huntington-beac...,For COVID safety I ordered for takeout. They a...,5,2021-06-13 19:06:26,"{'id': '2McpPoj2lrdpx1nbEsCV2A', 'profile_url'...",4jo2ikVD9jQ1zRdwSKgEpQ
2,Z0aogEeyM9iXI6lz-Wo_Fg,https://www.yelp.com/biz/dukes-huntington-beac...,I came here with friends on a Saturday afterno...,4,2021-05-09 17:53:02,"{'id': 'J2IxIf59M8vLpbeA2I8kiw', 'profile_url'...",kABAIb4nZQJOjBYbXAPpxw
3,ElvZ-9U8y_LtJvVQ3oUibg,https://www.yelp.com/biz/heirloom-a-modern-far...,Omg I have been meaning to get here forever. S...,5,2020-08-15 07:28:35,"{'id': 'v6dKCaMuJ1hIKnCAdoQteg', 'profile_url'...",Jtf9JrTlRL67AGbdt-aNZg
4,ElvZ-9U8y_LtJvVQ3oUibg,https://www.yelp.com/biz/heirloom-a-modern-far...,My wife and I tried a take out dinner and I ha...,5,2020-05-28 10:20:00,"{'id': 'FODGUELgcSHYheJR5yY25Q', 'profile_url'...",RM5s9-qWLnCyaIs-fa2rpA


In [13]:
#Copy dataframe data_review into dataframe data_review_all without id column
data_reviews_all = data_reviews.iloc[:,1:7]
data_reviews_all.head()


,url,text,rating,time_created,user,id_reviews
0,https://www.yelp.com/biz/dukes-huntington-beac...,I have never had a better bar tender than Jame...,5,2021-04-26 16:34:28,"{'id': '9oWEPTd5dI36u1Zf6_ONlw', 'profile_url'...",EeQ1jhQtEV3dMIPGP4cU6w
1,https://www.yelp.com/biz/dukes-huntington-beac...,For COVID safety I ordered for takeout. They a...,5,2021-06-13 19:06:26,"{'id': '2McpPoj2lrdpx1nbEsCV2A', 'profile_url'...",4jo2ikVD9jQ1zRdwSKgEpQ
2,https://www.yelp.com/biz/dukes-huntington-beac...,I came here with friends on a Saturday afterno...,4,2021-05-09 17:53:02,"{'id': 'J2IxIf59M8vLpbeA2I8kiw', 'profile_url'...",kABAIb4nZQJOjBYbXAPpxw
3,https://www.yelp.com/biz/heirloom-a-modern-far...,Omg I have been meaning to get here forever. S...,5,2020-08-15 07:28:35,"{'id': 'v6dKCaMuJ1hIKnCAdoQteg', 'profile_url'...",Jtf9JrTlRL67AGbdt-aNZg
4,https://www.yelp.com/biz/heirloom-a-modern-far...,My wife and I tried a take out dinner and I ha...,5,2020-05-28 10:20:00,"{'id': 'FODGUELgcSHYheJR5yY25Q', 'profile_url'...",RM5s9-qWLnCyaIs-fa2rpA


### Loading Yelp reviews into MongoDB

In [14]:
#Insert reviews into MongoDb
#We retrieve 3 reviews for each restaurant

key_business = "O111111111111111aaaaaA"


for ind, row in data_reviews.iterrows():
           
    if key_business !=  row['id']:
        key_business =  row['id']
        dict_reviews_all=dict(data_reviews_all.iloc[ind])
        db.business.update_one(
           { 'id': key_business },
           { '$set': {"reviews_yelp":[dict_reviews_all]}}
        )
        
    else:
        dict_reviews_all=dict(data_reviews_all.iloc[ind])
        db.business.update_one(
           { 'id': key_business },
           { '$push': {"reviews_yelp":dict_reviews_all}}
        )
        
    
print (f"Number of total reviews inserted:{ind+1}")       
     

Number of total reviews inserted:256


## Google API call to retrieve restaurant data


In [15]:
#Retrieve 20 at time. The first call is without the token parameter.
# geocoordinates
target_coordinates = "33.69173830884756,  -118.01170349121094"
target_radius = 10000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    'sensor':False,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()


In [16]:
#Retrieve 20 at time. The second call is with the token parameter retrieved on the first time.
time.sleep(5) 
# set up a parameters dictionary
target_next_page_token=response.json()["next_page_token"]
params2 = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    'sensor':False,
    "key": gkey,
    "pagetoken":target_next_page_token
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response2 = requests.get(base_url, params=params2)
places_data2 = response2.json()


In [17]:
#Retrieve 20 at time. The third call is with the token parameter retrieved on the second time.
time.sleep(5)
# set up a parameters dictionary
target_next_page_token=response2.json()["next_page_token"]
params3 = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    'sensor':False,
    "key": gkey,
    "pagetoken":target_next_page_token
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response3 = requests.get(base_url, params=params3)
places_data3 = response3.json()


In [18]:
#Insert data retrieved into 3 dataframes
df=pd.DataFrame(places_data['results'])
df2=pd.DataFrame(places_data2['results'])
df3=pd.DataFrame(places_data3['results'])

In [19]:
#Concatenate the dataframes
#Reset the index
df_row = pd.concat([df,df2,df3])
df_row = df_row.reset_index(drop=True)
df_row.head()

,business_status,geometry,icon,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity,permanently_closed
0,OPERATIONAL,"{'location': {'lat': 33.7341503, 'lng': -117.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,Buca di Beppo Italian Restaurant,{'open_now': False},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJP5s8IkIm3YARpSuO--EX3f8,"{'compound_code': 'P2M4+MV Huntington Beach, C...",2.0,4.1,ChIJP5s8IkIm3YARpSuO--EX3f8,GOOGLE,"[restaurant, food, point_of_interest, establis...",1443,"7979 Center Avenue, Huntington Beach",NaN
1,OPERATIONAL,"{'location': {'lat': 33.6907124, 'lng': -117.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,Hooters of Costa Mesa,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ6_qtJVTf3IARdsI3DGEHn_c,"{'compound_code': 'M3RH+7H Costa Mesa, CA, USA...",2.0,4.0,ChIJ6_qtJVTf3IARdsI3DGEHn_c,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",898,"1507 South Coast Drive, Costa Mesa",NaN
2,OPERATIONAL,"{'location': {'lat': 33.65664439999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,Duke's Huntington Beach,{'open_now': False},"[{'height': 933, 'html_attributions': ['<a hre...",ChIJg6yePEIh3YAR9jvzwIJzPoA,"{'compound_code': 'MX4X+M5 Huntington Beach, C...",2.0,4.5,ChIJg6yePEIh3YAR9jvzwIJzPoA,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",3391,"317 Pacific Coast Highway, Huntington Beach",NaN
3,OPERATIONAL,"{'location': {'lat': 33.732545, 'lng': -117.99...",https://maps.gstatic.com/mapfiles/place_api/ic...,The Cheesecake Factory,{'open_now': False},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJaRncL0Mm3YARs9osub8nU1s,"{'compound_code': 'P2M5+29 Huntington Beach, C...",2.0,4.3,ChIJaRncL0Mm3YARs9osub8nU1s,GOOGLE,"[restaurant, meal_takeaway, food, point_of_int...",1828,"7871 Edinger Avenue, Huntington Beach",NaN
4,OPERATIONAL,"{'location': {'lat': 33.7296127, 'lng': -117.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,BJ's Restaurant & Brewhouse,{'open_now': True},"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJ2ynLWlsm3YARmvBfFo6_vMQ,"{'compound_code': 'P2H6+RJ Huntington Beach, C...",2.0,4.1,ChIJ2ynLWlsm3YARmvBfFo6_vMQ,GOOGLE,"[restaurant, bar, food, point_of_interest, est...",1421,"16060 Beach Boulevard, Huntington Beach",NaN


### Transforming the Google data

In [20]:
#Drop the duplicates
df_row.drop_duplicates(subset ="name",keep = False, inplace = True)

In [21]:
#Removed some columns
#Reset the index
df_row = df_row.drop(['geometry','icon','photos','plus_code','reference','permanently_closed'], axis=1)
df_row = df_row.reset_index(drop=True)
df_row.head()

,business_status,name,opening_hours,place_id,price_level,rating,scope,types,user_ratings_total,vicinity
0,OPERATIONAL,Buca di Beppo Italian Restaurant,{'open_now': False},ChIJP5s8IkIm3YARpSuO--EX3f8,2.0,4.1,GOOGLE,"[restaurant, food, point_of_interest, establis...",1443,"7979 Center Avenue, Huntington Beach"
1,OPERATIONAL,Hooters of Costa Mesa,{'open_now': True},ChIJ6_qtJVTf3IARdsI3DGEHn_c,2.0,4.0,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",898,"1507 South Coast Drive, Costa Mesa"
2,OPERATIONAL,Duke's Huntington Beach,{'open_now': False},ChIJg6yePEIh3YAR9jvzwIJzPoA,2.0,4.5,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",3391,"317 Pacific Coast Highway, Huntington Beach"
3,OPERATIONAL,The Cheesecake Factory,{'open_now': False},ChIJaRncL0Mm3YARs9osub8nU1s,2.0,4.3,GOOGLE,"[restaurant, meal_takeaway, food, point_of_int...",1828,"7871 Edinger Avenue, Huntington Beach"
4,OPERATIONAL,Shin-Sen-Gumi Hakata Ramen - Fountain Valley,{'open_now': False},ChIJ93pCsTAn3YARYnvc3DKUUkk,1.0,4.6,GOOGLE,"[restaurant, food, point_of_interest, establis...",1152,"18315 Brookhurst Street #1, Fountain Valley"


### Using Google data to search MongoDB to find restaurants that match and don't match 

In [22]:
#Google restaurants found in MongoDb are stored in restaurants_found_df dataframe 

#Google restaurants not found in MongoDb are stored in restaurants_not_found_df dataframe

cols = ['place_id','name','opening_hours','price_level','rating','scope','types','user_ratings_total','location','business_status']
restaurants_found_df = pd.DataFrame(columns=cols)
restaurants_not_found_df = pd.DataFrame(columns=cols)

for ind, row in df_row.iterrows():
    #Search MondoDB by restaurant name to see if the restaurant exist
    return_data = db.business.find_one({"name":row['name']})
    if return_data != None:
        my_row = {"place_id":row['place_id'],
                  "name":row['name'],
                  "opening_hours":row['opening_hours'],
                  "price_level":row['price_level'],
                  "rating":row['rating'],
                  "scope":row['scope'],
                  "types":row['types'],
                  "user_ratings_total":row['user_ratings_total'],
                  "location":row['vicinity'],
                  "business_status":row['business_status']
                 }
        restaurants_found_df=restaurants_found_df.append(my_row, ignore_index=True)
        my_row.clear()
    else:
        my_row = {"place_id":row['place_id'],
                  "name":row['name'],
                  "opening_hours":row['opening_hours'],
                  "price_level":row['price_level'],
                  "rating":row['rating'],
                  "scope":row['scope'],
                  "types":row['types'],
                  "user_ratings_total":row['user_ratings_total'],
                  "location":row['vicinity'],
                  "business_status":row['business_status']
                 }
        restaurants_not_found_df=restaurants_not_found_df.append(my_row, ignore_index=True)
        my_row.clear()
        
        

In [23]:
restaurants_found_df.head()

,place_id,name,opening_hours,price_level,rating,scope,types,user_ratings_total,location,business_status
0,ChIJg6yePEIh3YAR9jvzwIJzPoA,Duke's Huntington Beach,{'open_now': False},2.0,4.5,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",3391,"317 Pacific Coast Highway, Huntington Beach",OPERATIONAL
1,ChIJRQNtE1An3YAR9z5rD7BMCog,Panda Express,{'open_now': False},1.0,4.1,GOOGLE,"[restaurant, food, point_of_interest, establis...",284,"18011 Newhope Street, Suite #a, Fountain Valley",OPERATIONAL
2,ChIJObOlBu4m3YAR-0rVhzIBaVc,The Donuttery,{'open_now': True},1.0,4.6,GOOGLE,"[bakery, cafe, restaurant, food, point_of_inte...",2123,"17420 Beach Boulevard, Huntington Beach",OPERATIONAL
3,ChIJ65KCLMQm3YARosGxAiwpyIc,Five Guys,{'open_now': False},2.0,4.3,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",369,"18645 Beach Boulevard, Huntington Beach",OPERATIONAL
4,ChIJt-CEYago3YARUgQByD9srjM,Popeyes Louisiana Kitchen,{'open_now': False},1.0,3.7,GOOGLE,"[restaurant, food, point_of_interest, establis...",492,"6948 Westminster Boulevard, Westminster",OPERATIONAL


In [24]:
restaurants_not_found_df.head()

,place_id,name,opening_hours,price_level,rating,scope,types,user_ratings_total,location,business_status
0,ChIJP5s8IkIm3YARpSuO--EX3f8,Buca di Beppo Italian Restaurant,{'open_now': False},2.0,4.1,GOOGLE,"[restaurant, food, point_of_interest, establis...",1443,"7979 Center Avenue, Huntington Beach",OPERATIONAL
1,ChIJ6_qtJVTf3IARdsI3DGEHn_c,Hooters of Costa Mesa,{'open_now': True},2.0,4.0,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",898,"1507 South Coast Drive, Costa Mesa",OPERATIONAL
2,ChIJaRncL0Mm3YARs9osub8nU1s,The Cheesecake Factory,{'open_now': False},2.0,4.3,GOOGLE,"[restaurant, meal_takeaway, food, point_of_int...",1828,"7871 Edinger Avenue, Huntington Beach",OPERATIONAL
3,ChIJ93pCsTAn3YARYnvc3DKUUkk,Shin-Sen-Gumi Hakata Ramen - Fountain Valley,{'open_now': False},1.0,4.6,GOOGLE,"[restaurant, food, point_of_interest, establis...",1152,"18315 Brookhurst Street #1, Fountain Valley",OPERATIONAL
4,ChIJ71mZpion3YARs_yxBkAyBt8,Fitzgerald's Irish Pub,{'open_now': True},1.0,4.4,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",211,"19171 Magnolia Street #12, Huntington Beach",OPERATIONAL


### Loading Google data into MongoDB

In [25]:
#Update MongoDB with the restaurants that Google found but yelp did not find previously

# Define the 'restaurantes_db' database in Mongo
db = client.restaurants_db

# Declare the collections
business = db.business

records = json.loads(restaurants_not_found_df.T.to_json()).values()
db.business.insert_many(records)




### Google API call to retrieve Reviews for restaurants previouly found by Yelp

In [26]:
#Search Google Reviews for Restaurantes Found In the Yelp Data
#Data is stored in data_google dataframe

cols = ['name', 'author_name', 'author_url', 'rating','relative_time_description','text','time']
data_google = pd.DataFrame(columns=cols)

for index,row in restaurants_found_df.iterrows():
    place = row['place_id']
    target_placeid = place
    target_fields = "name,rating,vicinity,reviews"
    #set up a parameters dictionary
    params = {
        "place_id": target_placeid,
        "fields": target_fields,
        "key": gkey}

    #GOOGLE URL for GET requests to retrieve restaurant data
    url="https://maps.googleapis.com/maps/api/place/details/json"
    # run a request using our params dictionary
    response = requests.get(url, params=params)
    restaurants_data=response.json()
 
    name = restaurants_data['result']['name']
    try:
        for resp in restaurants_data['result']['reviews']:
            my_row = {"name": name,
                      "author_name":resp['author_name'],
                      "author_url":resp['author_url'],
                      "rating":resp['rating'],
                      "relative_time_description":resp['relative_time_description'],
                      "text":resp['text'],
                      "time":resp['time']}
            data_google=data_google.append(my_row, ignore_index=True)
            my_row.clear()

    except (KeyError, IndexError):
            print("Missing field/reviews... skipping.")
        


In [27]:
data_google.head()

,name,author_name,author_url,rating,relative_time_description,text,time
0,Duke's Huntington Beach,Goyette Marable,https://www.google.com/maps/contrib/1174376508...,5,3 months ago,Duke's in located next to the Huntington Beach...,1614739869
1,Duke's Huntington Beach,Alessia Vaughn,https://www.google.com/maps/contrib/1159563230...,5,4 months ago,"We were here last Sunday, as we celebrated our...",1611859779
2,Duke's Huntington Beach,Skip Stephenson,https://www.google.com/maps/contrib/1180471949...,5,a month ago,Always a delight to dine at Duke's. The food i...,1619415676
3,Duke's Huntington Beach,James Bagette,https://www.google.com/maps/contrib/1050280493...,5,2 months ago,The food is very delicious. Everything on the ...,1617935894
4,Duke's Huntington Beach,Henri Fikkee,https://www.google.com/maps/contrib/1076662399...,5,3 months ago,"Great food, atmosphere, views, and service! Lo...",1615522463


In [28]:
#Copy dataframe data_google into dataframe data_google_all without name column
data_google_all = data_google.iloc[:,1:7]


In [29]:
data_google_all.head()
                    

,author_name,author_url,rating,relative_time_description,text,time
0,Goyette Marable,https://www.google.com/maps/contrib/1174376508...,5,3 months ago,Duke's in located next to the Huntington Beach...,1614739869
1,Alessia Vaughn,https://www.google.com/maps/contrib/1159563230...,5,4 months ago,"We were here last Sunday, as we celebrated our...",1611859779
2,Skip Stephenson,https://www.google.com/maps/contrib/1180471949...,5,a month ago,Always a delight to dine at Duke's. The food i...,1619415676
3,James Bagette,https://www.google.com/maps/contrib/1050280493...,5,2 months ago,The food is very delicious. Everything on the ...,1617935894
4,Henri Fikkee,https://www.google.com/maps/contrib/1076662399...,5,3 months ago,"Great food, atmosphere, views, and service! Lo...",1615522463


### Loading goggle reviews for the restaurants found in yelp

In [30]:
#Insert google reviews into MongoDb

name_business = "A11111111111aaaaaA"

for ind, row in data_google.iterrows():
    if name_business !=  row['name']:
        name_business =  row['name']
        dict_google_all=dict(data_google_all.iloc[ind])
        db.business.update_one(
            { 'name': name_business },
            { '$set': {"reviews_google":[dict_google_all]}}
        )
        
    else:
        dict_google_all=dict(data_google_all.iloc[ind])
        db.business.update_one(
            { 'name': name_business },
            { '$push': {"reviews_google":dict_google_all}}
        )
        


###  Google API call to retrieve Reviews for restaurants not found previouly by Yelp

In [31]:
#Search Google Reviews for Restaurantes NOT Found In the Yelp Data
#Data is stored in data_google_2 dataframe

cols = ['name', 'author_name', 'author_url', 'rating','relative_time_description','text','time']
data_google_2 = pd.DataFrame(columns=cols)


for index,row in restaurants_not_found_df.iterrows():
    place = row['place_id']
    target_placeid = place
    target_fields = "name,rating,vicinity,reviews"
    #set up a parameters dictionary
    params = {
        "place_id": target_placeid,
        "fields": target_fields,
        "key": gkey}

    #GOOGLE URL for GET requests to retrieve restaurant data
    url="https://maps.googleapis.com/maps/api/place/details/json"
    # run a request using our params dictionary
    response = requests.get(url, params=params)
    restaurants_data=response.json()
 
 
    name = restaurants_data['result']['name']

    try:
        for resp in restaurants_data['result']['reviews']:
            my_row = {"name": name,
                      "author_name":resp['author_name'],
                      "author_url":resp['author_url'],
                      "rating":resp['rating'],
                      "relative_time_description":resp['relative_time_description'],
                      "text":resp['text'],
                      "time":resp['time']}
            data_google_2=data_google_2.append(my_row, ignore_index=True)
            my_row.clear()

    except (KeyError, IndexError):
            print("Missing field/reviews... skipping.")
        


In [32]:
data_google_2.head()

,name,author_name,author_url,rating,relative_time_description,text,time
0,Buca di Beppo Italian Restaurant,Maria T. Unger,https://www.google.com/maps/contrib/1152512162...,1,in the last week,We had spaghetti with meatballs and Chicken Ma...,1623075393
1,Buca di Beppo Italian Restaurant,Kasandra Hood,https://www.google.com/maps/contrib/1029270637...,5,4 months ago,Connor was an excellent host. His personality...,1613256085
2,Buca di Beppo Italian Restaurant,Bill Reitz,https://www.google.com/maps/contrib/1162013550...,5,2 months ago,Good food served family style. Employees were ...,1616822779
3,Buca di Beppo Italian Restaurant,Leslie Elle,https://www.google.com/maps/contrib/1072422614...,5,in the last week,Best Italian food I’ve ever tasted If you ever...,1623583372
4,Buca di Beppo Italian Restaurant,Mark Chamberlain,https://www.google.com/maps/contrib/1070773848...,5,3 months ago,Great tasting food. Everything I had was delic...,1614657848


In [33]:
#Copy dataframe data_google_2 into dataframe data_google_all_2 without name column
data_google_all_2 = data_google_2.iloc[:,1:7]


In [34]:
data_google_all_2.head()

,author_name,author_url,rating,relative_time_description,text,time
0,Maria T. Unger,https://www.google.com/maps/contrib/1152512162...,1,in the last week,We had spaghetti with meatballs and Chicken Ma...,1623075393
1,Kasandra Hood,https://www.google.com/maps/contrib/1029270637...,5,4 months ago,Connor was an excellent host. His personality...,1613256085
2,Bill Reitz,https://www.google.com/maps/contrib/1162013550...,5,2 months ago,Good food served family style. Employees were ...,1616822779
3,Leslie Elle,https://www.google.com/maps/contrib/1072422614...,5,in the last week,Best Italian food I’ve ever tasted If you ever...,1623583372
4,Mark Chamberlain,https://www.google.com/maps/contrib/1070773848...,5,3 months ago,Great tasting food. Everything I had was delic...,1614657848


### Loading Google reviews for the restaurants previosly not found by Yelp

In [35]:
#Insert google reviews into MongoDb
name_business = "A11111111111aaaaaA"

for ind, row in data_google_2.iterrows():
    if name_business !=  row['name']:
        name_business =  row['name']
        dict_google_all_2=dict(data_google_all_2.iloc[ind])
        db.business.update_one(
            { 'name': name_business },
            { '$set': {"reviews_google":[dict_google_all_2]}}
        )
        
    else:
        dict_google_all_2=dict(data_google_all_2.iloc[ind])
        db.business.update_one(
            { 'name': name_business },
            { '$push': {"reviews_google":dict_google_all_2}}
        )
        
print(f"Update = {ind+1}")

Update = 180
